In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import requests
import time
from tqdm import trange
from bs4 import BeautifulSoup as bs

headers = {"user-agent": "Mozilla/5.0"}

def get_url(item_code, page_no=1):
    url = f"https://finance.naver.com/item/board.nhn"
    url = f"{url}?code={item_code}&page={page_no}"
    return url

def get_one_page(item_code, page_no):
    "한 페이지 수집"
    # 종목 URL 만들기
    url = get_url(item_code, page_no)
    # requests
    response = requests.get(url, headers=headers)
    soup = bs(response.text, 'lxml')

    # 테이블에서 각 행(row) 추출
    rows = soup.select('table.type2 > tbody > tr')
    
    data = []
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 1:  # 유효한 행만 선택
            date = columns[0].text.strip()
            title = columns[1].text.strip()
            author = columns[2].text.strip()
            view = columns[3].text.strip()
            upvotes = columns[4].text.strip()
            downvotes = columns[5].text.strip()

            data.append([date, title, author, view, upvotes, downvotes])
    
    # 데이터프레임으로 변환
    df = pd.DataFrame(data, columns=["날짜", "제목", "작성자", "조회", "공감", "비공감"])
    
    return df

def get_last_page(item_code):
    url = get_url(item_code)
    response = requests.get(url, headers=headers)
    soup = bs(response.text, 'lxml')
    last_page = int(soup.select("#content > div.section.inner_sub > table > tbody > tr > td > table > tbody > tr > td.pgRR > a")[-1]['href'].split('=')[-1])
    return last_page

def get_all_pages(item_code, start_page=1, end_page=None):
    "지정된 페이지 범위만 수집"
    if end_page is None:  # end_page가 설정되지 않았다면 마지막 페이지까지
        end_page = get_last_page(item_code)
    
    page_list = []
 
    # 지정된 범위의 페이지 수집
    for page_num in trange(start_page, end_page + 1):
        page = get_one_page(item_code, page_num)
        page_list.append(page)
        time.sleep(0.1)
    
    # 모든 페이지 하나의 데이터프레임으로 합치기
    df_all_page = pd.concat(page_list)
    # 조회, 공감, 비공감 열에서 숫자가 아닌 값을 NaN으로 처리하고, NaN 값을 포함한 행 제거
    df_all_page['조회'] = pd.to_numeric(df_all_page['조회'], errors='coerce')
    df_all_page['공감'] = pd.to_numeric(df_all_page['공감'], errors='coerce')
    df_all_page['비공감'] = pd.to_numeric(df_all_page['비공감'], errors='coerce')
    df_all_page = df_all_page.dropna(subset=['조회', '공감', '비공감'])

    # 인덱스 리셋
    df_all_page = df_all_page.reset_index(drop=True)

    return df_all_page

# 사용 예시
item_code = '005380'  # 현대차
start_page = int(input("시작 페이지를 입력하세요: "))
end_page = int(input("종료 페이지를 입력하세요: "))

# 지정한 범위의 페이지에서만 크롤링 수행
df_all = get_all_pages(item_code, start_page=start_page, end_page=end_page)
df_all.to_excel('bionote.xlsx', index=False)


시작 페이지를 입력하세요:  134
종료 페이지를 입력하세요:  3000


100%|██████████████████████████████████████████████████████████████████████████████| 2867/2867 [07:55<00:00,  6.02it/s]
